In [22]:
import json
import os
import pandas as pd
import requests

In [23]:
# setup file system
os.makedirs('data', exist_ok=True)

In [24]:
# get players
players = requests.get('https://api.sleeper.app/v1/players/nfl').json()

with open('data/players.json', 'w') as f:
    json.dump(players, f, indent=4)

In [25]:
# process players file
players_df = pd.read_json('data/players.json').transpose()[['first_name', 'last_name', 'position', 'team', 'status', 'injury_status', 'active', 'player_id']]

# filter players
players_df = players_df[players_df['position'].isin(['QB', 'RB', 'WR', 'TE', 'K', 'DEF'])]
players_df = players_df[players_df['active']]

# project players
players_df['name'] = players_df['first_name'] + " " + players_df['last_name']
players_df = players_df[['name', 'position', 'team', 'status', 'injury_status', 'player_id']]
players_df.head()

,name,position,team,status,injury_status,player_id
5870,Daniel Jones,QB,NYG,Active,None,5870
6343,Manny Wilkins,QB,None,Active,None,6343
8595,Kyric McGowan,WR,WAS,Active,None,8595
3199,Michael Thomas,WR,NO,Inactive,Out,3199
6487,Charles Scarff,TE,None,Inactive,None,6487


In [26]:
# player weekly stats for 2022
def get_weekly_stats(season, num_weeks):
    season_stats = []
    for week in range(1, num_weeks + 1):
        weekly = requests.get(f'https://api.sleeper.com/stats/nfl/{season}/{week}?season_type=regular&position[]=DEF&position[]=K&position[]=QB&position[]=RB&position[]=TE&position[]=WR').json()
        season_stats.extend(weekly) 
    return season_stats

season = 2022
weeks = 6
season_stats = get_weekly_stats(season, weeks)
with open(f'data/weekly-{season}.json', 'w') as f:
    json.dump(season_stats, f, indent=4)

In [28]:
# load season weekly stats
with open(f'data/weekly-{season}.json') as f:
    weekly = json.load(f)

weekly_df = pd.json_normalize(weekly).drop(columns=[
    'team', 'sport', 'season_type', 'company', 'category', 'game_id', 'date',
    'player.years_exp', 'player.news_updated', 'player.metadata', 'player.injury_status', 'player.injury_start_date', 'player.injury_notes', 'player.injury_body_part', 'player.fantasy_positions'
])
weekly_df.head()

,week,season,player_id,opponent,last_modified,stats.tm_st_snp,stats.tm_off_snp,stats.tm_def_snp,stats.gms_active,stats.adp_dd_ppr,...,stats.pts_idp,stats.idp_ff,stats.idp_tkl_ast,stats.tkl_solo_misc,stats.ff_misc,stats.yds_allow_550p,player.metadata.years_exp_shift,stats.st_ff,stats.idp_tkl_loss,stats.idp_safe
0,1,2022,1029,HOU,NaN,24.0,92.0,70.0,1.0,999.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2022,1034,LAC,NaN,21.0,58.0,67.0,1.0,999.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2022,1049,NYG,NaN,23.0,65.0,60.0,1.0,999.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2022,1067,WAS,NaN,17.0,70.0,77.0,1.0,999.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2022,1099,NYG,NaN,23.0,65.0,60.0,1.0,999.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
